In [ ]:
import torch
import os
import esm
import pandas as pd
import biotite.structure.io as bsio
import numpy as np
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model = esm.pretrained.esmfold_v1()
model = model.eval().cuda()

seq_list = pd.read_csv('/geniusland/home/wanglijuan/sci_proj/ESM2_opt/src_datasets/case1.csv')['Sequence'].tolist()
# Optionally, uncomment to set a chunk size for axial attention. This can help reduce memory.
# Lower sizes will have lower memory requirements at the cost of increased speed.
rs1 = []
rs2 = []
rs3 = []

# model.set_chunk_size(128)
id = 1

for pos in ['seq_100.csv', 'seq_300.csv', 'seq_500.csv']:
    pth = '/geniusland/home/wanglijuan/sci_proj/ESM2_opt/AMP_Methods_Compare/case3/MCTS/' + pos
    otp = '/geniusland/home/wanglijuan/sci_proj/ESM2_opt/AMP_Methods_Compare/pLDDT/c3/mcts/' + pos
    id = 1
    rs = []
    seq_list = pd.read_csv(pth)['Sequence'].tolist()
    for sequence in seq_list:
        with torch.no_grad():
            output = model.infer_pdb(sequence)
        with open(f"/geniusland/home/wanglijuan/sci_proj/ESM2_opt/AMP_Methods_Compare/pLDDT/c3/{id}_{sequence[0]}{sequence[-1]}-{len(sequence)}.pdb", "w") as f:
            f.write(output)
        struct = bsio.load_structure(f"/geniusland/home/wanglijuan/sci_proj/ESM2_opt/AMP_Methods_Compare/pLDDT/c3/{id}_{sequence[0]}{sequence[-1]}-{len(sequence)}.pdb", extra_fields=["b_factor"])
        id += 1
        # print(struct)
        rs.append(struct.b_factor.mean())
    df = pd.DataFrame(rs, columns=['Values'])
    df.to_csv(otp, index=False)
    
for pos in ['noprob_v1_seq_100.csv', 'noprob_v1_seq_300.csv', 'noprob_v1_seq_500.csv']:
    pth = '/geniusland/home/wanglijuan/sci_proj/ESM2_opt/AMP_Methods_Compare/case3/MCTS_esm2/' + pos
    otp = '/geniusland/home/wanglijuan/sci_proj/ESM2_opt/AMP_Methods_Compare/pLDDT/c3/mcts_esm2/' + pos
    id = 1
    rs = []
    seq_list = pd.read_csv(pth)['Sequence'].tolist()
    for sequence in seq_list:
        with torch.no_grad():
            output = model.infer_pdb(sequence)
        with open(f"/geniusland/home/wanglijuan/sci_proj/ESM2_opt/AMP_Methods_Compare/pLDDT/c3/{id}_{sequence[0]}{sequence[-1]}-{len(sequence)}.pdb", "w") as f:
            f.write(output)
        struct = bsio.load_structure(f"/geniusland/home/wanglijuan/sci_proj/ESM2_opt/AMP_Methods_Compare/pLDDT/c3/{id}_{sequence[0]}{sequence[-1]}-{len(sequence)}.pdb", extra_fields=["b_factor"])
        id += 1
        # print(struct)
        rs.append(struct.b_factor.mean())
    df = pd.DataFrame(rs, columns=['Values'])
    df.to_csv(otp, index=False)